In [1]:
import requests
import json
import datetime

In [2]:
def r_to_j(r):
    myJ_s = ''
    line_num=1
    for n in r:
        #print(line_num, n)
        myJ_s += n.decode("'iso-8859-1'")
        
#    print(myJ_s)
    myJ = json.loads(myJ_s)
#    print(json.dumps(myJ_s, indent=1))
    return myJ


In [20]:
def do_r(req, page_size=2, page_num=1,debug=False, attrs=[]):
    myJ = {}
    rJ = {'count': 0, 'g' : {}}
    keepGoing=1

    req += '&page_size='+str(page_size)+'&page_num='

    while keepGoing:
    
        page_num += 1
        if page_num > 0:
            keepGoing = 0
        
        r = requests.get(req + str(page_num) )
        if debug:
            print(r, page_num, datetime.datetime.now())
        
        myJ = r_to_j(r)
        print('myj=', json.dumps(myJ, indent=1))
        if len(myJ['feed']['entry']) == 0:
            keepGoing=0
            continue

        rJ['count']+= len(myJ['feed']['entry'])
        outJ={}
        for j in myJ['feed']['entry']:
            outJ={}
            for a in attrs:
                if a in j:
                # Major Hack!
                    if a == 'links':
                        myL = []
                        for links in j[a]:
                            for l in links:
                                #print ('l=', links[l])
                                if l == 'href':
                                    myL.append({'href':links[l]})
                        #print ('myL=', myL)
                        j[a]=myL
                    outJ[a]=j[a]
                # Major Hack!
                
            rJ['g'][j['id']]=outJ
                
                    
#            for k in j:
#                if k in uniqueKeysJ:
#                    uniqueKeysJ[k]['cnt']+=1
#                    if len(str(j[k])) <= 505:
#                        if j[k] in uniqueKeysJ[k]['values']:
#                            continue
#                        else:
#                            uniqueKeysJ[k]['values'].append(j[k])
#                
#                else:
#                    uniqueKeysJ[k]= {'cnt':1}
#                    if len(str(j[k])) <= 505:
#                        uniqueKeysJ[k]['values'] = [j[k]]
#                    else:
#                        uniqueKeysJ[k]['values'] = ['So Long!']

#    for k in uniqueKeysJ:
#        print (k, uniqueKeysJ[k]['cnt'], end=' ')
#        if len(uniqueKeysJ[k]['values']) <= 16:
#            print (uniqueKeysJ[k]['values'])
#        else:
#            print (len(uniqueKeysJ[k]['values']), 'unique values')

    return rJ

In [27]:
# Get a list of short_names from a provider

my_provider='LAADS'
my_shortNames=[]
#print (r.headers)
myJ_s = ''
collectionsJ={}
uniqueKeysJ={}

page_num=0
keepGoing=1

while keepGoing:
    
    page_num += 1
    if page_num > 200:
        keepGoing = 0
        
    #r = requests.get('https://cmr.earthdata.nasa.gov/search/collections.umm_json?platform=AQUA&sort_key=short_name&page_size=2&page_num='+str(page_num))
    #r = requests.get('https://cmr.earthdata.nasa.gov/search/collections.umm_json?provider=GES_DISC&sort_key=short_name&page_size=2000&page_num='+str(page_num))
    #r = requests.get('https://cmr.earthdata.nasa.gov/search/collections.umm_json?instrument=MODIS&INSTRUMENT=MODIS&provider=LPDAAC_ECS&sort_key=short_name&page_size=2&page_num='+str(page_num))
    r = requests.get('https://cmr.earthdata.nasa.gov/search/collections.json?&provider=LAADS&sort_key=short_name&page_size=2000&page_num='+str(page_num))

    myJ_s = ''
    line_num=1
    for n in r:
        #print(line_num, n)
        myJ_s += n.decode("'iso-8859-1'")
        
#    print(myJ_s)
    myJ = json.loads(myJ_s)
    #print(json.dumps(myJ, indent=1))

    if len(myJ['items']) == 0:
        keepGoing=0
        continue
    
    for r in myJ['items']:
        j=r['meta']
        #print ('j=', j)
        #print ('r=', json.dumps(r))
        #for k in j:
        #    print ('meta',k, j[k])
        j=r['umm']
#        for k in j:
            #print ('umm', k)
#            if k == 'ShortName':
        my_shortNames.append({'ShortName' : j['ShortName'],
                                     'native-id' : r['meta']['native-id'],
                                'EntryTitle' : j['EntryTitle']})
        if j['ShortName']=='MYD021KM':
            print (json.dumps(r,indent=1))

i=1
for s in my_shortNames:
    print (i, s)
    i+=1
#print (my_shortNames)

KeyError: 'items'

In [28]:
#r_base = 'https://cmr.earthdata.nasa.gov/search/granules.json?provider_id=LAADS&short_name='
r_base = 'https://cmr.earthdata.nasa.gov/search/granules.json?provider_id=LAADS&short_name='
uniqueKeysJ={}    
i=1
my_shortNames = [{'ShortName' :'MYD021KM'}]
#my_shortNames = [{'ShortName' :'cosmirimpacts'}]
for s in my_shortNames:
    #print (i, s)
    i+=1

    r = r_base + s['ShortName']

    gJ = do_r(r, attrs=['granule_size','links'],debug=True)
    #print ('# Granules', gJ['count'])
    
    gn=0
    granuleSizeTotal=0.0
    
    urls = {}
    
    for g in gJ['g']:
        gn+=1
        gm=gJ['g'][g]
        #print(gn,g)
        #print(json.dumps(gm['granule_size']))
        #print ('x', json.dumps(gm))
        
        if 'granule_size' in gm:
            pass
        else:
            gm['granule_size']=0.0
            
        granuleSizeTotal += float(gm['granule_size'])
        
        ln=0
        if 'links' in gm:
            for l in gm['links']:
                for k in l:
                    if k == 'href':
                        ln+=1
                        #print ('...', ln, l[k])
                        turl=l[k].replace('https://','').split('/')
                        #print (turl)
                        url=turl[0]
                        #print (url)
                        if url in urls:
                            urls[url]+=1
                        else:
                            urls[url]=1
                            
    if gn > 0:
            print(s['ShortName'] + '~' + '#grans=' + str(gn) + '~' + 'granuleSizeTotal(MB)=' + str(round( granuleSizeTotal, 3)) + '~' + json.dumps(urls))
    else:
            print(s['ShortName'] + '~' + '#grans=' + str(gn) + '~' + 'granuleSizeTotal(MB)=' + str(round( granuleSizeTotal, 3)))




<Response [200]> 2 2021-01-15 17:23:16.477475
myj= {
 "feed": {
  "updated": "2021-01-15T17:23:16.454Z",
  "id": "https://cmr.earthdata.nasa.gov:443/search/granules.json?provider_id=LAADS&short_name=MYD021KM&page_size=2&page_num=2",
  "title": "ECHO granule metadata",
  "entry": [
   {
    "producer_granule_id": "MYD021KM.A2002185.0005.006.2012032175624.hdf",
    "time_start": "2002-07-04T00:05:00.000Z",
    "updated": "2019-04-10T00:17:22.201Z",
    "dataset_id": "MODIS/Aqua Calibrated Radiances 5-Min L1B Swath 1km V006",
    "data_center": "LAADS",
    "title": "LAADS:786850069",
    "coordinate_system": "GEODETIC",
    "day_night_flag": "NIGHT",
    "time_end": "2002-07-04T00:10:00.000Z",
    "id": "G237201409-LAADS",
    "original_format": "ECHO10",
    "granule_size": "63.1926612854004",
    "browse_flag": true,
    "polygons": [
     [
      "11.923153 12.1398 8.978899 33.318102 26.674192 38.429938 30.093362 14.821938 11.923153 12.1398"
     ]
    ],
    "collection_concept_id": 

In [ ]:
https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/6/MYD021KM/2002/185/MYD021KM.A2002185.0005.006.2012032175624.hdf"
https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/61/MYD021KM/2002/185/MYD021KM.A2002185.0005.061.2017362175129.hdf